<a href="https://colab.research.google.com/github/SolovevDR/NIR/blob/main/ParserAccsessLog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка среды для работы(запускать строго в этом порядке, не будет иначе geoip работать)

In [1]:
pip install python-geoip-geolite2

In [2]:
pip install python-geoip-python3

In [3]:
pip install user_agents

In [4]:
pip install apachelogs

In [5]:
pip install datetime

In [6]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/NIR')



---
---
---



In [7]:
import time 
import pandas as pd
from urllib.parse import urlparse
import re
from dateutil.parser import parse as dateparser
from user_agents import parse
from geoip import geolite2
import datetime
from apachelogs import LogParser
from nltk.stem.porter import PorterStemmer

In [8]:
def ban_separation_date_time(datetim):
    req_requset = re.compile(r'([0-9]+.[0-9]+.[0-9]+).([0-9]+:[0-9]+:[0-9]+.[0-9]+).(.[0-9]+)')
    date_and_time_list = []
    for i in range(len(datetim)):
        result = req_requset.match(datetim[i])
        date_and_time = result.group(1) + ' ' + result.group(2) + result.group(3)

        dt = dateparser(date_and_time)
        date_and_time_list.append(dt)
    return  date_and_time_list

a = ban_separation_date_time(['2021-10-05 22:51:16.972634 +0000 UTC'])

In [9]:
lem_word = []

In [10]:
def create_dict(new_date_list, status_list, ip_list):
    count_word_in_dict = 0
    ban_dict = {}
    minus_time = datetime.timedelta(seconds=30)
    for i in range(len(new_date_list)):
        add_time = new_date_list[i] - minus_time
        if (status_list[i] == 'added'):
            if ban_dict.get(ip_list[i]) == None: 
                ban_dict[ip_list[i]] = [add_time]
                count_word_in_dict += 1
            else: 
                spis = ban_dict[ip_list[i]]
                spis.append(add_time)
                ban_dict[ip_list[i]] = spis
                count_word_in_dict += 1
        elif (status_list[i] == 'removed') and (ban_dict.get(ip_list[i]) == None):
            b = add_time
            b = b - datetime.timedelta(minutes=1)
            ban_dict[ip_list[i]] = [b, add_time]
            count_word_in_dict += 1
        if (status_list[i] == 'removed') and (ban_dict.get(ip_list[i]) != None):
                spis = ban_dict[ip_list[i]]
                spis.append(new_date_list[i])
                ban_dict[ip_list[i]] = spis
                count_word_in_dict += 1
    keys_list = list(ban_dict.keys())
    for i in keys_list:
        time_ban_unban = ban_dict.get(i)
        if len(time_ban_unban)%2 != 0:
            b = time_ban_unban[len(time_ban_unban)-1]
            b = b + datetime.timedelta(seconds=30)
            spis = ban_dict[i]
            spis.append(b)
            ban_dict[i] = spis
            count_word_in_dict += 1
    print('values in dict - ',count_word_in_dict)
    return(ban_dict)


In [11]:
def read_dict(ban_dict):
    list_key_dict = list(ban_dict.keys())
    for i in list_key_dict:
        if len(list(ban_dict.get(i))) % 2 != 0:
          print(len(list(ban_dict.get(i))), '-', i)

In [12]:
def find_info_with_ip(ip):

    m = geolite2.lookup(ip)

    try:
        country = m.country
    except:
        country = "None"

    try:
        continent = m.continent
    except:
        continent = "None"

    try:
        timezone = m.timezone
    except:
        timezone = "None"

    return country, continent, timezone

country, continent, timezone = find_info_with_ip('17.0.0.1')

In [13]:
def separation_date_time(datetim):
    #req_requset = re.compile(r'.([0-9]+.[0-9]+.[0-9]+).([0-9]+:[0-9]+:[0-9]+)(.[0-9]+.[0-9]+)')

    #result = req_requset.match(datetime)
    #date_and_time = result.group(1) + ' ' + result.group(2)

    dt = dateparser(datetim)
    new_datetime = dt.astimezone()
    return  new_datetime, #new_datetime.date(), new_datetime.time()

new_datetime = separation_date_time('2019-01-22 03:56:14+03:30')

In [14]:
def find_info_with_request(request):

    url_path = None
    number_availability = 0
    statuses = False
    del_list = []
    
    try:
        method, url, http = request.split()

        url_line = urlparse(url)

        if (url_line.path != '') and (url_line.params != '') and (url_line.query != ''):
            url_path = url_line.path + " " + url_line.params + " " + url_line.query
        elif (url_line.path != '') and (url_line.params != ''):
            url_path = url_line.path + " " + url_line.params
        elif (url_line.path != '') and (url_line.params):
            url_path = url_line.path + " " + url_line.querty
        elif (url_line.params != '') and (url_line.query != ''):
            url_path = url_line.path + " " + url_line.query
        elif (url_line.path != ''):
            url_path = url_line.path
        elif (url_line.params != ''):
            url_path = url_line.params
        elif (url_line.query != ''):
            url_path = url_line.query
        else:
            url_path = None
        

        if (url_path != None) and (url_path != '/'):
            url_path = url_path.replace(',', " ").replace('.', " ").replace('/', " ").replace('%', " ").replace('&', " ").replace('*', " ").replace('=', " ").replace('_', " ").replace('-', " ")
            url_path = " ".join(url_path.split())
            url_path = url_path.split(" ")
            for i in url_path:
                if i == '':
                    url_path.remove(i)
                s = re.search('\d+', i) is not None
                if s == True:
                    del_list.append(i)
                    if number_availability == 0:
                        number_availability = 1   #назвать столбик в дататесте так 
            for i in range(len(del_list)):
                url_path.remove(del_list[i])
            if len(url_path) != 0:
                statuses = True
                url_path = " ".join(url_path)
                url_path = url_path.split(" ")
                len_url = len(url_path)
                if len_url != 0:
                    for i in range(len_url):
                        new_str = url_path[i][0].upper() + url_path[i][1:]
                        url_path[i] = new_str
                        spis = re.findall('[A-Z][^A-Z]*', url_path[i])
                        for j in spis:
                            url_path.append(j)
                    for i in range(len_url):
                        url_path.pop(0)
                    url_path = " ".join(url_path)
                    url_path = url_path.lower()
        elif url_path == '/':
            url_path = 'None'

        if statuses == False:
            url_path = 'None'
            
                
                
            

    except Exception as ex:
        print(ex, '-', request)
        method = 'None'
        url_path = 'None'
        http = 'None'

    

    return method, url_path, number_availability, http

#find_info_with_request('GET /image/60844/productModel/200x200 HTTP/1.1')
find_info_with_request('GET /DDoS01/3f5300a4/netdike/f0fe037bb78055683440c6c77bdca1f2Lw== HTTP/1.0')


('GET', 'netdike', 1, 'HTTP/1.0')

In [15]:
def find_info_with_Useragent(useragent_line):
    req_request = re.compile(r'([a-zA-z]+).')

    user_agent = parse(useragent_line)

    try:
        os = user_agent.os.family
    except:
        os = 'other'

    try:
        browser = user_agent.browser.family
    except:
        browser = 'other'

    try:
        device = user_agent.device.family
    except:
        device = 'other'

    try:
        result = req_request.match(useragent_line)
        exit_system = result.group(1)
    except:
        exit_system = 'other'

    return exit_system, os, browser, device

find_info_with_Useragent('Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36')

('Mozilla', 'Android', 'Chrome Mobile', 'ALE-L21')

In [16]:
def copy_size_return_object(size_object):


    try:
        new_size_return_object = int(size_object)
    except:
        new_size_return_object = 0

    return new_size_return_object

copy_size_return_object(5667)

5667

In [17]:
df = pd.read_csv('ban_unban_table.csv')

date_list = df['created_at'].tolist()
status_list = df['action'].tolist()
ip_list = df['ip'].tolist()

new_date_list = ban_separation_date_time(date_list)

In [18]:
chek = []

In [19]:
def markup(datetim, ip, ban_dict, number):
    keys_list = list(ban_dict.keys())
    if ip in keys_list:
        spis = ban_dict[ip]
        if len(spis) != 0:
            i = 0
            datetim = datetim[0]
            while i != len(spis):
                ban_time = spis[i]
                unban_time = spis[i+1]
                if (datetim > ban_time) and (datetim < unban_time):
                    return 0 
                else:
                    i = i + 2
        else:
            return 1   
    else:
        return 1


In [20]:
#col_names = ['country', 'continent', 'timezone', 'date', 'time', 'method', 'url_sheme', 'url_netloc', 'url_path', 'url_params', 'url_querty', 'url_fragment', 'http', 'status_code', 'size_object', 'exit_system', 'os', 'browser', 'device']
my_df = pd.DataFrame()


In [21]:
ban_dict = create_dict(new_date_list, status_list, ip_list)

values in dict -  45328


In [22]:
read_dict(ban_dict)

In [23]:
parser2 = LogParser('%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"')

ip_list = []
country_list = []
continent_list = []
timezone_list = []
# date_list = []
# time_list = []
datetime_list = []
method_list = []
url_path_list = []
number_availability_list = []
http_list = []
size_return_object_list = []
exit_system_list = []
os_list = []
browser_list = []
device_list = []
status_code_list = []
status_request_list = []
number = 0

date_time_obj_start = (datetime.datetime(2021, 11, 3, 11, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(0), 'UTC')),)
date_time_obj_end = (datetime.datetime(2021, 11, 3, 12, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(0), 'UTC')),)


with open('srvgame.ru.access.log') as fp:  # doctest: +SKIP
    for entry in parser2.parse_lines(fp):
        datetim = separation_date_time(str(entry.directives['%t']))
        if (datetim[0] > date_time_obj_start[0]) and (datetim[0] < date_time_obj_end[0]):
            country, continent, timezone = find_info_with_ip(str(entry.remote_host))
            method, url_path, number_availability, http = find_info_with_request(str(entry.request_line))
            status_code = entry.directives['%>s']
            size_return_object = copy_size_return_object(entry.directives['%b'])
            exit_system, os, browser, device = find_info_with_Useragent(str(entry.directives['%{User-Agent}i']))
            mark = markup(datetim, str(entry.remote_host), ban_dict, number)


            ip_list.append(str(entry.remote_host))
            status_code_list.append(status_code)
            country_list.append(country)
            continent_list.append(continent)
            timezone_list.append(timezone)
            # date_list.append(date)
            # time_list.append(time)
            datetime_list.append(datetim)
            method_list.append(method)
            url_path_list.append(url_path)
            number_availability_list.append(number_availability)
            http_list.append(http)
            size_return_object_list.append(size_return_object)
            exit_system_list.append(exit_system)
            os_list.append(os)
            browser_list.append(browser)
            device_list.append(device)
            status_request_list.append(mark)
            number = number + 1

my_df['ip'] = ip_list
my_df['country'] = country_list
my_df['continent'] = continent_list
my_df['timezone'] = timezone_list
my_df['datetime'] = datetime_list
# my_df['date'] = date_list
# my_df['time'] = time_list
my_df['method'] = method_list
my_df['url_path'] = url_path_list
my_df['number_availability'] = number_availability_list
my_df['http'] = http_list
my_df['status_code'] = status_code_list
my_df['size_object'] = size_return_object_list
my_df['exit_system'] = exit_system_list
my_df['os'] = os_list
my_df['browser'] = browser_list
my_df['device'] = device_list
my_df['markup'] = status_request_list
my_df.to_csv('DICT_FOR_LEARN.csv', header=True, sep=';')

del ip_list, country_list, continent_list, timezone_list, datetime_list, method_list, url_path_list, number_availability_list, http_list, status_code_list, size_return_object_list
del exit_system_list, os_list, browser_list, device_list, status_request_list, number

In [24]:
my_df['markup'].value_counts()

1.0    176505
0.0      6376
Name: markup, dtype: int64